
# Transformers を使用した大規模な事前トレーニング

:label: `sec_large-pretraining-transformers`

これまでの画像分類と機械翻訳の実験では、特定のタスクを実行するために、入出力サンプルを使用してデータセット上でモデルを*ゼロから*トレーニングしました。たとえば、Transformer は英語とフランス語のペア (:numref: `sec_transformer` ) でトレーニングされたため、このモデルは入力された英語テキストをフランス語に翻訳できます。その結果、各モデルは、データ分布のわずかな変化 (:numref: `sec_environment-and-distribution-shift` ) にも敏感な*特定のエキスパート*になります。より優れた一般化モデル、または適応の有無にかかわらず複数のタスクを実行できる有能な*ジェネラリストを*実現するには、大規模なデータでモデルを*事前トレーニングすること*がますます一般的になってきています。

事前トレーニング用に大きなデータが与えられると、Transformer アーキテクチャはモデル サイズとトレーニング コンピューティングの増加によりパフォーマンスが向上し、優れた*スケーリング*動作を示します。具体的には、Transformer ベースの言語モデルのパフォーマンスは、モデル パラメーター、トレーニング トークン、トレーニング コンピューティングの量に応じてべき乗則としてスケールします :cite: `kaplan2020scaling` 。 Transformer のスケーラビリティは、より大きなデータでトレーニングされた、より大きなビジョン Transformer によるパフォーマンスの大幅な向上によっても証明されています (:numref: `sec_vision-transformer`で説明)。最近の成功例としては、Atari の再生、キャプション画像、チャット、ロボットとして機能する*ジェネラリスト*モデルである Gato が挙げられます:cite: `reed2022generalist` 。 Gato は、テキスト、画像、関節トルク、ボタンの押下など、さまざまなモダリティで事前トレーニングされた場合に適切に拡張できる単一の Transformer です。特に、このようなマルチモーダル データはすべて、トークンのフラット シーケンスにシリアル化され、トランスフォーマーによってテキスト トークン (:numref: `sec_transformer` ) または画像パッチ (:numref: `sec_vision-transformer` ) と同様に処理できます。

マルチモーダル データの Transformer の事前トレーニングが確実に成功する前に、Transformer は豊富なテキストを使用して広範囲に事前トレーニングされました。もともと機械翻訳用に提案された :numref: `fig_transformer`の Transformer アーキテクチャは、入力シーケンスを表すエンコーダーとターゲット シーケンスを生成するデコーダーで構成されます。主に、Transformer は、 *encoder-only* 、 *encoder-decoder* 、および*decoder-only*の 3 つの異なるモードで使用できます。この章の締めくくりとして、これら 3 つのモードを確認し、Transformer の事前トレーニングにおけるスケーラビリティについて説明します。

## エンコーダのみ

Transformer エンコーダのみが使用される場合、入力トークンのシーケンスは、出力 (分類など) にさらに投影できる同じ数の表現に変換されます。 Transformer エンコーダーは、すべての入力トークンが相互に注意を払うセルフアテンション レイヤーで構成されます。たとえば、:numref: `fig_vit`で示されているビジョン トランスフォーマーはエンコーダ専用であり、入力イメージ パッチのシーケンスを特別な「&lt;cls&gt;」の表現に変換します。トークン。この表現はすべての入力トークンに依存するため、さらに分類ラベルに投影されます。この設計は、テキストで事前トレーニングされた以前のエンコーダ専用の Transformer からインスピレーションを得ました: BERT (Bidirectional Encoder Representations from Transformers) :cite: `Devlin.Chang.Lee.ea.2018` 。

###  BERT の事前トレーニング

![](../img/bert-encoder-only.svg) :label: `fig_bert-encoder-only`

 BERT は、*マスクされた言語モデリング*を使用してテキスト シーケンスで事前トレーニングされます。ランダムにマスクされたトークンを含む入力テキストは、マスクされたトークンを予測するために Transformer エンコーダーに供給されます。 :numref: `fig_bert-encoder-only`に示すように、元のテキスト シーケンス「I」、「love」、「this」、「red」、「car」の先頭に「&amp;lt;cls&amp;gt;」が付加されます。トークンと「&lt;マスク&gt;」トークンはランダムに「love」を置き換えます。その場合、マスクされたトークン「love」とその予測の間のクロスエントロピー損失は、事前トレーニング中に最小限に抑えられます。 Transformer エンコーダーのアテンション パターン ( :numref: `fig_bert-encoder-only`の右側) には制約がないため、すべてのトークンが相互にアテンションできることに注意してください。したがって、「愛」の予測は、シーケンス内のその前後の入力トークンに依存します。これが、BERT が「双方向エンコーダ」である理由です。手動でラベルを付ける必要がなく、書籍や Wikipedia からの大規模なテキスト データを BERT の事前トレーニングに使用できます。

###  BERTの微調整

事前トレーニングされた BERT は、単一のテキストまたはテキストのペアを含むダウンストリームのエンコード タスクに*合わせて微調整*できます。微調整中に、ランダム化されたパラメーターを使用して追加のレイヤーを BERT に追加できます。これらのパラメーターと事前トレーニングされた BERT パラメーターは、下流タスクのトレーニング データに合わせて*更新*されます。 

![](../img/bert-finetune-classification.svg) :label: `fig_bert-finetune-classification`

 :numref: `fig_bert-finetune-classification`感情分析のための BERT の微調整を示しています。 Transformer エンコーダは事前トレーニングされた BERT であり、テキスト シーケンスを入力として受け取り、「&amp;lt;cls&amp;gt;」を供給します。表現 (入力のグローバル表現) を追加の完全に接続されたレイヤーに追加してセンチメントを予測します。微調整中、感情分析データの予測とラベルの間のクロスエントロピー損失は、勾配ベースのアルゴリズムによって最小限に抑えられます。このアルゴリズムでは、BERT の事前トレーニング済みパラメーターが更新される間に、追加のレイヤーが最初からトレーニングされます。 BERT は感情分析以上のことを行います。 2,500 億のトレーニング トークンから 3 億 5,000 万のパラメータの BERT によって学習された一般言語表現は、単一テキストの分類、テキスト ペアの分類または回帰、テキストのタグ付け、質問応答などの自然言語タスクの最先端を進歩させました。

これらの下流タスクにはテキスト ペアの理解が含まれることに注意してください。 BERT の事前トレーニングでは、ある文が別の文の直後に続くかどうかを予測する際に、別の損失が発生します。ただし、この損失は、同じサイズの BERT バリアントである RoBERTa を 20000 億トークンで事前トレーニングする場合には役に立たないことが後に判明しました:cite: `liu2019roberta` 。 BERT の他の派生モデルでは、ALBERT (パラメーター共有の強制) :cite: `lan2019albert` 、 SpanBERT (テキストのスパンの表現と予測) :cite: `joshi2020spanbert` 、 DistilBERT (知識の蒸留による軽量化) :cite: `sanh2019distilbert`など、モデル アーキテクチャまたは事前トレーニング目標が改善されました。および ELECTRA (トークン検出の置き換え) :cite: `clark2019electra` 。さらに、BERT は、ビジョン Transformers :cite: `Dosovitskiy.Beyer.Kolesnikov.ea.2021` 、Swin Transformers :cite: `liu2021swin` 、および MAE (masked autoencoders) :cite: `he2022masked`など、コンピューター ビジョンにおける Transformer の事前トレーニングに影響を与えました。

## エンコーダ-デコーダ

Transformer エンコーダは入力トークンのシーケンスを同じ数の出力表現に変換するため、エンコーダ専用モードでは機械翻訳のような任意の長さのシーケンスを生成できません。もともと機械翻訳用に提案されていたように、Transformer アーキテクチャには、エンコーダ出力とデコーダ出力の両方を条件として、任意の長さのターゲット シーケンスをトークンごとに自己回帰的に予測するデコーダを装備できます。(i) エンコーダ出力の条件付けの場合、エンコーダデコーダのクロスアテンション ( :numref: `fig_transformer`のデコーダのマルチヘッド アテンション) により、ターゲット トークンが*すべての*入力トークンに対応できるようになります。 (ii) デコーダ出力の条件付けは、いわゆる*因果的*注意 (この名前は文献では一般的ですが、因果関係の適切な研究とはほとんど関係がないため誤解を招きます) パターン (以下のデコーダのマスクされたマルチヘッド アテンション) によって実現されます。 numref: `fig_transformer` )、ここで、ターゲット トークンはターゲット シーケンス内の*過去*と*現在の*トークンのみに対応します。

人間がラベル付けした機械翻訳データを超えてエンコーダー デコーダー トランスフォーマーを事前トレーニングするために、BART :cite: `lewis2019bart`と T5 :cite: `raffel2020exploring`は、大規模なテキスト コーパスで事前トレーニングされた 2 つのエンコーダー デコーダー トランスフォーマーとして同時に提案されています。どちらも事前トレーニングの目的で元のテキストを再構築しようとしますが、前者はノイズ入力（マスキング、削除、順列、回転など）を強調し、後者は包括的なアブレーション研究によるマルチタスクの統合を強調します。

### 事前トレーニング T5

事前トレーニング済みの Transformer エンコーダ/デコーダの例として、T5 (Text-to-Text Transfer Transformer) は、多くのタスクを同じテキスト間問題として統合します。どのタスクでも、エンコーダの入力はタスクの説明です (例: "Summarize"、":") の後にタスク入力 (記事からのトークンのシーケンスなど) が続き、デコーダはタスクの出力 (入力記事を要約するトークンのシーケンスなど) を予測します。 text-to-text として実行するために、T5 は入力テキストを条件としてターゲット テキストを生成するようにトレーニングされています。 

![](../img/t5-encoder-decoder.svg) :label: `fig_t5-encoder-decoder`

元のテキストから入力と出力を取得するために、T5 は連続するスパンを予測するように事前トレーニングされています。具体的には、テキストのトークンが特別なトークンにランダムに置き換えられ、連続する各スパンが同じ特別なトークンに置き換えられます。 :numref: `fig_t5-encoder-decoder`の例を考えてみましょう。ここで、元のテキストは「I」、「love」、「this」、「red」、「car」です。 「love」、「red」、「car」のトークンはランダムに特別なトークンに置き換えられます。 「赤」と「車」は連続したスパンであるため、同じ特殊トークンに置き換えられます。その結果、入力シーケンスは「I」、「&lt;X&gt;」、「this」、「&lt;Y&gt;」、ターゲットシーケンスは「&lt;X&gt;」、「love」、「&lt;Y&gt;」になります。 ;"、"赤"、"車"、"&lt;Z&gt;"、"&lt;Z&gt;"これも終わりを告げる特別なトークンです。 :numref: `fig_t5-encoder-decoder`に示すように、デコーダーには、シーケンス予測中に将来のトークンに注意を払わないようにするための因果的注意パターンがあります。

 T5 では、連続するスパンの予測は破損したテキストの再構築とも呼ばれます。この目的を達成するために、T5 は、Web :cite: `raffel2020exploring`のクリーンな英語テキストで構成される C4 (Colossal Clean Crawled Corpus) データからの 10,000 億トークンを使用して事前トレーニングされています。

###  T5 の微調整

BERT と同様に、このタスクを実行するには、タスク固有のトレーニング データに基づいて T5 を微調整する (T5 パラメーターを更新する) 必要があります。 BERT 微調整との主な違いは次のとおりです。(i) T5 入力にはタスクの説明が含まれます。 (ii) T5 は、Transformer デコーダを使用して任意の長さのシーケンスを生成できます。 (iii) 追加の層は必要ありません。 

![](../img/t5-finetune-summarization.svg) :label: `fig_t5-finetune-summarization`

 :numref: `fig_t5-finetune-summarization`テキスト要約を例として使用して T5 の微調整について説明します。このダウンストリーム タスクでは、タスク説明トークン「Summarize」、「:」に続いて記事トークンがエンコーダーに入力されます。

微調整後、110 億パラメータの T5 (T5-11B) は、複数のエンコード (分類など) および生成 (要約など) ベンチマークで最先端の結果を達成しました。 T5 はリリースされて以来、その後の研究で広く使用されてきました。たとえば、スイッチ トランスフォーマーは、計算効率を向上させるためにパラメーターのサブセットをアクティブにするために T5 に基づいて設計されています (引用: `fedus2022switch` )。 Imagen と呼ばれるテキストから画像へのモデルでは、テキストは 46 億のパラメーターを持つフリーズされた T5 エンコーダー (T5-XXL) に入力されます:cite: `saharia2022photorealistic` 。 :numref: `fig_imagen`のフォトリアリスティックなテキストから画像への例は、微調整しなくても T5 エンコーダーだけで効果的にテキストを表現できることを示唆しています。 

![](../img/imagen.png) :幅: `700px` :ラベル: `fig_imagen`

## デコーダのみ

エンコーダのみのトランスフォーマとエンコーダ - デコーダのトランスフォーマをレビューしました。あるいは、デコーダ専用のトランスフォーマは、 :numref: `fig_transformer`で示されている元のエンコーダ - デコーダ アーキテクチャから、エンコーダ全体と、エンコーダ - デコーダのクロスアテンションを持つデコーダ サブレイヤを削除します。現在、デコーダ専用の Transformers は、自己教師あり学習を介して世界中の豊富なラベルなしテキスト コーパスを活用する大規模言語モデリング (:numref: `sec_language-model` ) における事実上のアーキテクチャとなっています。

###  GPT と GPT-2

言語モデリングをトレーニング目標として使用し、GPT (生成事前トレーニング) モデルはバックボーンとして Transformer デコーダーを選択します (引用: `Radford.Narasimhan.Salimans.ea.2018` )。 

![](../img/gpt-decoder-only.svg) :label: `fig_gpt-decoder-only`

 :numref: `subsec_partitioning-seqs`で説明されている自己回帰言語モデルのトレーニングに続いて、 :numref: `fig_gpt-decoder-only` Transformer エンコーダーを使用した GPT 事前トレーニングを示しています。ターゲット シーケンスは、1 トークン分シフトされた入力シーケンスです。 Transformer デコーダのアテンション パターンにより、各トークンは過去のトークンのみに対応できるようになります (将来のトークンはまだ選択されていないため、対応できません)。

 GPT には 1 億のパラメータがあり、個々の下流タスクに合わせて微調整する必要があります。はるかに大きな Transformer デコーダ言語モデルである GPT-2 が 1 年後に導入されました :cite: `Radford.Wu.Child.ea.2019` 。 GPT のオリジナルの Transformer デコーダと比較して、GPT-2 では事前正規化 (:numref: `subsec_vit-encoder`で説明) と改善された初期化と重みスケーリングが採用されました。 40 GB のテキストで事前トレーニングされた 15 億パラメータの GPT-2 は、*パラメータやアーキテクチャを更新することなく、*言語モデリング ベンチマークで最先端の結果を取得し、他の複数のタスクでも有望な結果を達成しました。

###  GPT-3

 GPT-2 は、モデルを更新せずに複数のタスクに同じ言語モデルを使用できる可能性を実証しました。これは、勾配計算によるモデルの更新が必要な微調整よりも計算効率が高くなります。 

![](../img/gpt-3-xshot.svg) :label: `fig_gpt-3-xshot`

パラメーターを更新せずに言語モデルをより計算的に効率的に使用する方法を説明する前に、言語モデルを`sec_rnn-scratch` 、プレフィックス テキスト シーケンスを条件としたテキスト シーケンスを生成できることを思い出してください。したがって、事前トレーニングされた言語モデルは、タスクの説明、タスク固有の入出力例、およびプロンプト (タスク入力) を含む入力シーケンスを条件として、*パラメーター更新のない*シーケンスとしてタスク出力を生成できます。この学習パラダイムは、*インコンテキスト学習*:cite: `brown2020language`と呼ばれます。これは、タスク固有の入出力例が存在しない、1 つまたは*少数*の場合、*ゼロショット**、*ワンショット、および少数ショットにさらに分類できます。 、それぞれ (:numref: `fig_gpt-3-xshot` )。 

![](../img/gpt3-xshot-scaling.png) :幅: `400px` :ラベル: `fig_gpt3-xshot-scaling`

これら 3 つの設定は GPT-3 :cite: `brown2020language`でテストされました。その最大のバージョンでは、GPT-2 よりも約 2 桁大きいデータとモデル サイズが使用されます。 GPT-3 は、アテンション パターン (:numref: `fig_gpt-decoder-only`の右側) が交互のレイヤーでまばらになることを除いて、その直接の前身である GPT-2 と同じ Transformer デコーダー アーキテクチャを使用します。 3,000 億のトークンで事前トレーニングされた GPT-3 は、モデル サイズが大きくなるとパフォーマンスが向上し、数ショットのパフォーマンスが最も急速に向上します (:numref: `fig_gpt3-xshot-scaling` )。

大規模な言語モデルは、テキスト入力を定式化して、コンテキスト内学習 (*プロンプト*とも呼ばれる) を介してモデルに目的のタスクを実行させるという刺激的な見通しを提供します。たとえば、*思考連鎖プロンプト*:cite: `wei2022chain`は、数ショットの「質問、中間推論ステップ、回答」のデモンストレーションを備えたコンテキスト内学習メソッドであり、数学的、常識的、常識的な問題を解決するための大規模な言語モデルの複雑な推論能力を引き出します。そして象徴的な推論タスク。複数の推論パスのサンプリング :cite: `wang2023self` 、少数ショットのデモンストレーションの多様化 :cite: `zhang2023automatic` 、および複雑な問題を部分問題に削減する :cite: `zhou2023least` 、すべて推論の精度を向上させることができます。実際、各回答の直前に「ステップごとに考えてみましょう」のような単純なプロンプトを使用すると、大規模な言語モデルは、*ゼロショットの*思考連鎖推論をかなりの精度で実行することもできます :cite: `kojima2022large` 。テキストと画像の両方で構成されるマルチモーダル入力の場合でも、言語モデルは、テキスト入力のみを使用する場合よりもさらに向上した精度でマルチモーダル思考連鎖推論を実行できます :cite: `zhang2023multicot` 。

## スケーラビリティ

:numref: `fig_gpt3-xshot-scaling` GPT-3 言語モデルにおける Transformer のスケーラビリティを実証的に示します。言語モデリングについては、Transformer のスケーラビリティに関するより包括的な実証研究により、研究者は、より多くのデータと計算を使用して大規模な Transformer をトレーニングすることが可能であると考えています :cite: `kaplan2020scaling` 。 

![](../img/scaling-power-law.png) :幅: `700px` :ラベル: `fig_scaling-power-law3`

 :numref: `fig_scaling-power-law3`に示すように、*べき乗則スケーリングは*、モデル サイズ (埋め込み層を除くパラメーターの数)、データセット サイズ (トレーニング トークンの数)、およびデータセットの量に関するパフォーマンスで観察できます。トレーニング計算 (PetaFLOP/秒日、埋め込み層を除く)。一般に、これら 3 つの要素をすべて同時に増加させると、パフォーマンスが向上します。ただし、それらを並行して増やす*方法については*依然として議論の余地があります:cite: `hoffmann2022training` 。 

![](../img/scaling-sample-conv.png) :幅: `700px` :ラベル: `fig_scaling-sample-conv`

パフォーマンスの向上に加えて、大型モデルは小型モデルよりも優れたサンプル効率も得られます。 :numref: `fig_scaling-sample-conv` 、大規模なモデルが小規模なモデルで達成されるのと同じレベルで実行するために必要なトレーニング サンプル (処理されるトークン) が少なく、パフォーマンスがコンピューティングによってスムーズにスケーリングされることを示しています。 

![](../img/scaling-gpt3.png) :幅: `400px` :ラベル: `fig_scaling-gpt3`

 :citet: `kaplan2020scaling`の経験的なスケーリング動作は、後続の大規模な Transformer モデルでテストされています。たとえば、GPT-3 は、 :numref: `fig_scaling-gpt3`でさらに 2 桁大きな値でこの仮説をサポートしました。

 GPT シリーズの Transformer のスケーラビリティは、その後の Transformer 言語モデルに影響を与えました。 GPT-3 の Transformer デコーダは主に OPT (Open Pretrained Transformers) に倣いました:cite: `zhang2022opt`二酸化炭素排出量は前者の 1/7 のみですが、GPT-2 Transformer デコーダは 5,300 億パラメータのトレーニングに使用されました。 Megatron-Turing NLG :cite: `smith2022using` 2,700 億のトレーニング トークンを使用。 GPT-2 設計に従って、3,000 億のトークンで事前トレーニングされた 2,800 億のパラメーター Gopher :cite: `rae2021scaling`は、約 150 の多様なタスクの大部分で最先端のパフォーマンスを達成しました。 Gopher と同じアーキテクチャを継承し、同じコンピューティング バジェットを使用する Chinchilla :cite: `hoffmann2022training`は、かなり小規模 (700 億パラメーター) のモデルで、はるかに長時間 (1 兆 4000 億のトレーニング トークン) をトレーニングし、多くのタスクで Gopher を上回ります。言語モデリングのスケーリング ラインを継続するために、7,800 億のトークンで事前トレーニングされた変更された設計を持つ 5,400 億パラメータの Transformer デコーダである PaLM (Pathway Language Model) :cite: `chowdhery2022palm`は、BIG-Bench ベンチマーク :cite: で人間の平均パフォーマンスを上回りました。 `srivastava2022beyond` 。科学的および数学的な内容を含む 385 億のトークンで PaLM をさらにトレーニングすると、物理学、化学、生物学などの定量的推論を必要とする学部レベルの問題の 3 分の 1 近くに答えることができる大規模な言語モデルである Minerva :cite: `lewkowycz2022solving`が生成されます。経済。

 :citet: `wei2022emergent`大規模な言語モデルにのみ存在し、小規模なモデルには存在しない、大規模な言語モデルの創発的な機能について説明しました。ただし、モデルのサイズを単に大きくしただけでは、本質的にモデルが人間の指示に従いやすくなるわけではありません。微調整 :cite: `ouyang2022training`を通じて言語モデルを人間の意図に合わせる InstructGPT に続き、 [ChatGPT は](https://chat.openai.com/)人間との会話から得られるコードのデバッグやメモの作成などの指示に従うことができます。

## 要約と考察

トランスフォーマーは、エンコーダー専用 (例: BERT)、エンコーダー-デコーダー (例: T5)、およびデコーダー専用 (例: GPT シリーズ) として事前トレーニングされています。事前トレーニングされたモデルは、モデル更新 (例: 微調整) またはそうでない (例: 少数のショット) を使用してさまざまなタスクを実行するように適合させることができます。 Transformer のスケーラビリティは、より大きなモデル、より多くのトレーニング データ、より多くのトレーニング コンピューティングによってパフォーマンスが向上することを示唆しています。 Transformers は最初にテキスト データ用に設計され、事前トレーニングされたため、このセクションは自然言語処理に少し傾いています。それにもかかわらず、上で説明したこれらのモデルは、複数のモダリティにわたるより最近のモデルでよく見られます。たとえば、(i) Chinchilla :cite: `hoffmann2022training` 、数ショット学習用の視覚言語モデルである Flamingo :cite: `alayrac2022flamingo`にさらに拡張されました。 (ii) GPT-2 :cite: `Radford.Wu.Child.ea.2019`とビジョン Transformer は、CLIP (Contrastive Language-Image Pre-training) でテキストと画像をエンコードします :cite: `radford2021learning` 。その画像とテキストの埋め込みは後に採用されました。 DALL-E 2 テキストから画像へのシステム:cite: `ramesh2022hierarchical`内。マルチモーダル事前トレーニングにおける Transformer のスケーラビリティに関する体系的な研究はまだありませんが、最近のすべて Transformer のテキストから画像へのモデルである Parti :cite: `yu2022scaling` 、モダリティ全体でのスケーラビリティの可能性を示しています。つまり、Parti が大きいほど、高いパフォーマンスが得られます。 -忠実な画像生成とコンテンツ豊富なテキストの理解 (:numref: `fig_parti` )。 

![](../img/parti.png) :幅: `700px` :ラベル: `fig_parti`

## 演習
1. さまざまなタスクで構成されるミニバッチを使用して T5 を微調整することは可能ですか?なぜ、あるいはなぜそうではないのでしょうか？ GPT-2の場合はどうでしょうか？
1. 強力な言語モデルがあれば、どのようなアプリケーションが考えられますか?
1. レイヤーを追加してテキスト分類を実行するように言語モデルを微調整するように求められたとします。どこに追加しますか?なぜ？
1. 入力シーケンスがターゲット シーケンスの予測全体を通じて常に利用できる場合のシーケンス間問題 (機械翻訳など) を考えてみましょう。デコーダーのみのトランスフォーマーを使用したモデリングの制限は何ですか?なぜ？

[ディスカッション](https://discuss.d2l.ai/t/9232)
